In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
import skimage.draw
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")


In [2]:
dataset = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/nuclei_segmentation/data/maskrcnn"
weights = "/root/Mask_RCNN/mask_rcnn_coco.h5"
logs = "/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/nuclei_segmentation/logs"
command = "train"

In [3]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR) 

from mrcnn import model as modellib, utils, visualize
from nuclei import NucleiDataset, NucleiConfig


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")


# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a 
    central point to control graph sizes.

    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax


Using TensorFlow backend.


In [4]:
def train(model, dataset):
    """Train the model."""
    # Training dataset.
    dataset_train = NucleiDataset()
    dataset_train.load_nucleus(dataset, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = NucleiDataset()
    dataset_val.load_nucleus(dataset, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=50,
                layers='heads')

    # Training - Stage 2
    # Finetune layers from ResNet stage 4 and up
    print("Fine tune Resnet stage 4 and up")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=120,
                layers='4+')

    # Training - Stage 3
    # Fine tune all layers
    print("Fine tune all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE / 10,
                epochs=160,
                layers='all')


def test(model, image_path):
    print("Running on {}".format(image_path))
    
    # Read image
    image = skimage.io.imread(image_path)

    # detect objects
    r = model.detect([image], verbose=1)[0]

    # Display results
    ax = get_ax(1)
    visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'],
                                ["Background", "IC"], r['scores'], ax=ax,
                                title="Predictions")



In [5]:
# Configurations
if command == "train":
    config = NucleiConfig()
else:
    class InferenceConfig(NucleiInferenceConfig):
        # Set batch size to 1 since we'll be running inference on
        # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    config = InferenceConfig()
config.display()

# Create model
if command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=logs)
else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=logs)

weights_path = weights

# Load weights
print("Loading weights ", weights_path)
# Exclude the last layers because they require a matching
# number of classes
model.load_weights(weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc",
    "mrcnn_bbox", "mrcnn_mask"])

# Train or evaluate
if command == "train":
    train(model, dataset)
elif command == "test":
    test(model, image_path=image)
else:
    print("'{}' is not recognized. "
          "Use 'train' or 'test'".format(command))

W0811 17:50:09.793133 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0811 17:50:09.820662 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0811 17:50:09.855787 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0811 17:50:09.885811 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:1919: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

W0811 17:50:09.890176 140122010


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

W0811 17:50:13.234850 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/backend/tensorflow_backend.py:2018: The name tf.image.resize_nearest_neighbor is deprecated. Please use tf.compat.v1.image.resize_nearest_neighbor instead.

W0811 17:50:14.106640 140122010900224 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0811 17:50:14.292889 140122010900224 deprecation_wrapper.py:119] From /root/Mask_RCNN/mrcnn/model.py:554: The name tf.random_shuffle is deprecated. Please use tf.random.shuffle instead.

W0811 17:50:14.565184 140122010900224 deprecation_wrapper.py:119] From /root/Mask_RCNN/mrcnn/utils.py:200: The name tf.log is deprecated. Please use tf.math.log instead.


Loading weights  /root/Mask_RCNN/mask_rcnn_coco.h5

nuclei

nuclei
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /allen/programs/celltypes/workgroups/em-connectomics/gayathrim/nuclei_segmentation/logs/nuclei20190811T1750/mask_rcnn_nuclei_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         

W0811 17:50:23.251007 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0811 17:50:28.934828 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0811 17:50:28.938004 140122010900224 deprecation_wrapper.py:119] From /usr/local/lib/python3.5/dist-packages/keras/callbacks.py:853: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



...

100/100 [==============================] - 2428s 24s/step - loss: 0.2797 - rpn_class_loss: 0.0034 - rpn_bbox_loss: 0.0506 - mrcnn_class_loss: 0.0424 - mrcnn_bbox_loss: 0.0357 - mrcnn_mask_loss: 0.1475 - val_loss: 1.7829 - val_rpn_class_loss: 0.0413 - val_rpn_bbox_loss: 0.3491 - val_mrcnn_class_loss: 0.2526 - val_mrcnn_bbox_loss: 0.2418 - val_mrcnn_mask_loss: 0.8980
Epoch 73/120
100/100 [==============================] - 2433s 24s/step - loss: 0.1727 - rpn_class_loss: 0.0031 - rpn_bbox_loss: 0.0387 - mrcnn_class_loss: 0.0155 - mrcnn_bbox_loss: 0.0209 - mrcnn_mask_loss: 0.0946 - val_loss: 1.4737 - val_rpn_class_loss: 0.0215 - val_rpn_bbox_loss: 0.3117 - val_mrcnn_class_loss: 0.1303 - val_mrcnn_bbox_loss: 0.2108 - val_mrcnn_mask_loss: 0.7993
Epoch 74/120
100/100 [==============================] - 2437s 24s/step - loss: 0.2368 - rpn_class_loss: 0.0019 - rpn_bbox_loss: 0.0305 - mrcnn_class_loss: 0.0239 - mrcnn_bbox_loss: 0.0328 - mrcnn_mask_loss: 0.1477 - val_loss: 1.3200 - val_rpn_class_lo

100/100 [==============================] - 2432s 24s/step - loss: 0.2372 - rpn_class_loss: 8.2930e-04 - rpn_bbox_loss: 0.0289 - mrcnn_class_loss: 0.0159 - mrcnn_bbox_loss: 0.0292 - mrcnn_mask_loss: 0.1624 - val_loss: 1.2611 - val_rpn_class_loss: 0.0168 - val_rpn_bbox_loss: 0.3132 - val_mrcnn_class_loss: 0.1117 - val_mrcnn_bbox_loss: 0.2340 - val_mrcnn_mask_loss: 0.5854
Epoch 94/120
100/100 [==============================] - 2434s 24s/step - loss: 0.2071 - rpn_class_loss: 0.0023 - rpn_bbox_loss: 0.0359 - mrcnn_class_loss: 0.0183 - mrcnn_bbox_loss: 0.0285 - mrcnn_mask_loss: 0.1221 - val_loss: 1.0186 - val_rpn_class_loss: 0.0059 - val_rpn_bbox_loss: 0.2531 - val_mrcnn_class_loss: 0.0359 - val_mrcnn_bbox_loss: 0.1717 - val_mrcnn_mask_loss: 0.5520
Epoch 95/120
100/100 [==============================] - 2430s 24s/step - loss: 0.2331 - rpn_class_loss: 6.1022e-04 - rpn_bbox_loss: 0.0395 - mrcnn_class_loss: 0.0191 - mrcnn_bbox_loss: 0.0377 - mrcnn_mask_loss: 0.1361 - val_loss: 1.3819 - val_rpn_

100/100 [==============================] - 2519s 25s/step - loss: 0.1783 - rpn_class_loss: 2.8812e-04 - rpn_bbox_loss: 0.0176 - mrcnn_class_loss: 0.0129 - mrcnn_bbox_loss: 0.0198 - mrcnn_mask_loss: 0.1276 - val_loss: 1.4377 - val_rpn_class_loss: 0.0119 - val_rpn_bbox_loss: 0.2634 - val_mrcnn_class_loss: 0.1558 - val_mrcnn_bbox_loss: 0.2167 - val_mrcnn_mask_loss: 0.7899
Epoch 115/120
100/100 [==============================] - 2468s 25s/step - loss: 0.1886 - rpn_class_loss: 5.6459e-04 - rpn_bbox_loss: 0.0232 - mrcnn_class_loss: 0.0128 - mrcnn_bbox_loss: 0.0216 - mrcnn_mask_loss: 0.1304 - val_loss: 1.5859 - val_rpn_class_loss: 0.0382 - val_rpn_bbox_loss: 0.3127 - val_mrcnn_class_loss: 0.1531 - val_mrcnn_bbox_loss: 0.2254 - val_mrcnn_mask_loss: 0.8564
Epoch 116/120
100/100 [==============================] - 2440s 24s/step - loss: 0.2388 - rpn_class_loss: 0.0020 - rpn_bbox_loss: 0.0369 - mrcnn_class_loss: 0.0282 - mrcnn_bbox_loss: 0.0274 - mrcnn_mask_loss: 0.1444 - val_loss: 1.8916 - val_rp

Epoch 121/160
100/100 [==============================] - 2806s 28s/step - loss: 0.6974 - rpn_class_loss: 0.0163 - rpn_bbox_loss: 0.1157 - mrcnn_class_loss: 0.0541 - mrcnn_bbox_loss: 0.1028 - mrcnn_mask_loss: 0.4084 - val_loss: 1.4121 - val_rpn_class_loss: 0.0135 - val_rpn_bbox_loss: 0.2765 - val_mrcnn_class_loss: 0.1977 - val_mrcnn_bbox_loss: 0.2669 - val_mrcnn_mask_loss: 0.6574
Epoch 122/160
100/100 [==============================] - 2773s 28s/step - loss: 0.4073 - rpn_class_loss: 0.0108 - rpn_bbox_loss: 0.0748 - mrcnn_class_loss: 0.0320 - mrcnn_bbox_loss: 0.0647 - mrcnn_mask_loss: 0.2250 - val_loss: 1.4390 - val_rpn_class_loss: 0.0039 - val_rpn_bbox_loss: 0.2019 - val_mrcnn_class_loss: 0.1032 - val_mrcnn_bbox_loss: 0.1725 - val_mrcnn_mask_loss: 0.9574
Epoch 123/160
100/100 [==============================] - 2745s 27s/step - loss: 0.4089 - rpn_class_loss: 0.0043 - rpn_bbox_loss: 0.0951 - mrcnn_class_loss: 0.0364 - mrcnn_bbox_loss: 0.0602 - mrcnn_mask_loss: 0.2130 - val_loss: 1.2008 - 

100/100 [==============================] - 2850s 29s/step - loss: 0.3383 - rpn_class_loss: 0.0016 - rpn_bbox_loss: 0.0814 - mrcnn_class_loss: 0.0310 - mrcnn_bbox_loss: 0.0459 - mrcnn_mask_loss: 0.1785 - val_loss: 1.6814 - val_rpn_class_loss: 0.0292 - val_rpn_bbox_loss: 0.4452 - val_mrcnn_class_loss: 0.1799 - val_mrcnn_bbox_loss: 0.1738 - val_mrcnn_mask_loss: 0.8534
Epoch 143/160
100/100 [==============================] - 2861s 29s/step - loss: 0.2962 - rpn_class_loss: 0.0022 - rpn_bbox_loss: 0.0548 - mrcnn_class_loss: 0.0222 - mrcnn_bbox_loss: 0.0421 - mrcnn_mask_loss: 0.1747 - val_loss: 1.4017 - val_rpn_class_loss: 0.0071 - val_rpn_bbox_loss: 0.2583 - val_mrcnn_class_loss: 0.2038 - val_mrcnn_bbox_loss: 0.2492 - val_mrcnn_mask_loss: 0.6832
Epoch 144/160
100/100 [==============================] - 2866s 29s/step - loss: 0.2774 - rpn_class_loss: 0.0027 - rpn_bbox_loss: 0.0479 - mrcnn_class_loss: 0.0275 - mrcnn_bbox_loss: 0.0380 - mrcnn_mask_loss: 0.1613 - val_loss: 1.3950 - val_rpn_class_